In [105]:
import numpy as np
import pandas as pd
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model
import keras
import zipfile
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
# set seed to reproduce similar results
np.random.seed(0)

In [106]:
# 데이터 디렉토리 설정
data_dir = "C:/Users/wdd45/OneDrive/바탕 화면/딥러닝응용/human/UCI HAR Dataset/UCI HAR Dataset"


In [107]:
# 데이터 로딩 및 전처리
def load_HAR_data(data_dir):
    # 훈련 데이터 로딩
    train_signals, train_labels, test_signals, test_labels = [], [], [], []
    
    for signal_type in ["train", "test"]:
        for signal in ["Inertial Signals/total_acc_x_", "Inertial Signals/body_acc_x_", "Inertial Signals/body_gyro_x_"]:
            filename = f"{data_dir}/{signal_type}/{signal}{signal_type}.txt"
            data = pd.read_csv(filename, delim_whitespace=True, header=None).values
            if "train" in signal_type:
                train_signals.append(data)
            else:
                test_signals.append(data)

        # 레이블 로딩
        labels_filename = f"{data_dir}/{signal_type}/y_{signal_type}.txt"
        labels = pd.read_csv(labels_filename, header=None, names=["Activity"]).values
        if "train" in signal_type:
            train_labels.append(labels)
        else:
            test_labels.append(labels)

    # 데이터를 NumPy 배열로 변환
    train_signals = np.transpose(np.array(train_signals), (1, 2, 0))
    train_labels = train_labels[0]
    test_signals = np.transpose(np.array(test_signals), (1, 2, 0))
    test_labels = test_labels[0]

    return train_signals, train_labels, test_signals, test_labels

In [119]:
# 데이터 로딩
train_signals, train_labels, test_signals, test_labels = load_HAR_data(data_dir)

# 레이블 수정 (0부터 시작하도록)
train_labels = train_labels - 1
test_labels = test_labels - 1

In [120]:
# 데이터 정규화
train_signals = (train_signals - np.mean(train_signals)) / np.std(train_signals)
test_signals = (test_signals - np.mean(test_signals)) / np.std(test_signals)

In [121]:
# 데이터를 훈련 및 검증 세트로 분할
X_train, X_val, y_train, y_val = train_test_split(train_signals, train_labels, test_size=0.2, random_state=42)

In [122]:
# 모델 아키텍처 정의
model = keras.Sequential([
    keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(6, activation='softmax')  # 클래스 수에 따라 조정
])

In [123]:
# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [124]:
# 모델 요약
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 126, 64)           640       
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 63, 64)           0         
 1D)                                                             
                                                                 
 flatten_8 (Flatten)         (None, 4032)              0         
                                                                 
 dense_16 (Dense)            (None, 128)               516224    
                                                                 
 dense_17 (Dense)            (None, 6)                 774       
                                                                 
Total params: 517,638
Trainable params: 517,638
Non-trainable params: 0
_______________________________________________

In [125]:
# 모델 훈련
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
184/184 [==============================] - ETA: 0s - loss: 0.5924 - accuracy: 0.7211WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x0000021085AE6828> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`

In [126]:
# 모델 평가
test_loss, test_accuracy = model.evaluate(test_signals, test_labels)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

93/93 [==============================] - 0s 2ms/step - loss: 0.3656 - accuracy: 0.8558
Test accuracy: 85.58%
